In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from matplotlib.pylab import plt
import seaborn as sns




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install nltk


In [ ]:
pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=8a9f53e595ab27f1d5dbd61275c5d9a650b14db8f75bbdf8560afe35b817fb74
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tugas SEM3/Pemrosesan Teks/UTS/WELFake_Dataset.csv')
df.head(5)

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
df.shape

(71537, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71576 non-null  object
 1   text    72095 non-null  object
 2   label   72134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [ ]:
y = df.label
print(f'Jumlah Data:\n')
y.value_counts().rename({0: 'real', 1: 'fake'})

Jumlah Data:



fake    37084
real    35028
Name: label, dtype: int64

In [ ]:
df.isnull().sum()

title         0
text          0
label         0
title_text    0
dtype: int64

In [ ]:
df.dropna(inplace=True)

## Menggabungkan kolom tittle dengan text

In [ ]:
df["title_text"] = df["title"] + df["text"]
df.iloc[0:4,:]

,title,text,label,title_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,NaN,Did they post their votes for Hillary already?,1,NaN
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."


In [ ]:
# df.title_text.dtype(str)
df.title_text.info

<class 'pandas.core.series.Series'>
RangeIndex: 72134 entries, 0 to 72133
Series name: title_text
Non-Null Count  Dtype 
--------------  ----- 
72134 non-null  object
dtypes: object(1)
memory usage: 563.7+ KB


## Preprocessing Text

In [ ]:
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub(r'\n{2,}', '', text)
    text = re.sub('\n', '', text) # Remove every '\n'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # Remove every url
    text = re.sub('rt', '', text) # Remove every retweet symbol
    text = re.sub('user', '', text) # Remove every username
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = re.sub(r'\S+@\S+', '', text)#Remove Email
    text = re.sub(r'(@\S+|#\S+)', '',text)#remove hastags and mention
    # text = re.sub(r'\d{1,2}(st|nd|rd|th)?[-./]\d{1,2}[-./]\d{2,4}', '', text) #remove tanggal like 22/03/2014
    # text = re.sub(r'(\d{1,2})?(st|nd|rd|th)?[-./,]?\s?(of)?\s?([J|j]an(uary)?|[F|f]eb(ruary)?|[Mm]ar(ch)?|[Aa]pr(il)?|[Mm]ay|[Jj]un(e)?|[Jj]ul(y)?|[Aa]ug(ust)?|[Ss]ep(tember)?|[Oo]ct(ober)?|[Nn]ov(ember)?|[Dd]ec(ember)?)\s?(\d{1,2})?(st|nd|rd|th)?\s?[-./,]?\s?(\d{2,4})?','',text)#remove tanggal like : 20 apr 21, April 15th, 11th of April, 2021
    text = re.sub(r'\b\w*[*]+\w*\b|\b\w+[*]+\b', '', text) #menghapus kalimat yang disensor
    return text

def numbers_to_words(text):
    # Mencari angka dalam teks menggunakan ekspresi reguler
    matches = re.findall(r'\d+', text)

    # Mengubah angka menjadi kata dan menggantinya dalam teks
    for match in matches:
        try:
            num = int(match)
            word = num2words(num, lang='en')
            text = text.replace(match, word)
        except ValueError:
            # Jika angka tidak dapat diubah menjadi kata, lewati saja
            continue

    return text

def remove_nonaplhanumeric(text):
    text = re.sub ('[^0-9a-zA-Z]+', ' ', text)
    return text

def remove_stopwords(text):
    STOPWORDS = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

stemmer = PorterStemmer()

def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])


In [ ]:
def preprocess(text):
    text = lowercase (text) # 1
    text = remove_unnecessary_char(text) #2
    text = remove_nonaplhanumeric(text)#3
    text = numbers_to_words(text)#4
    text = stem_words (text) #5
    text = remove_stopwords (text) #6

    return text


In [ ]:
import time

# Waktu awal
start_time = time.time()

# Menjalankan operasi yang ingin diukur waktunya
# df['title_text'] = df['title_text'].astype(str)
df['hasil_preprocessed_title_text'] = df['title_text'].apply(preprocess)

# Waktu berakhir
end_time = time.time()

# Menghitung waktu yang diperlukan
execution_time = end_time - start_time

print(f"Waktu eksekusi: {execution_time} detik")

Waktu eksekusi: 836.4836223125458 detik


In [ ]:
df.to_csv('/content/drive/MyDrive/Tugas SEM3/Kecerdasan Artifisial/Project AI/data_bersih_baru',index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tugas SEM3/Pemrosesan Teks/UTS/data_bersih.csv')
df.head(5)

,title_text,body_len,hasil_preprocessed_title_text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,4222,law enforc high ale follow threat cop white ni...,1
1,Did they post their votes for Hillary already?,39,post vote hillari alreadi,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,299,unbeliev obama attorney gener say charlott rio...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",6811,bobbi jindal rais hindu use stori christian co...,0
4,SATAN 2: Russia unvelis an image of its terrif...,1668,satan two russia unv imag terrifi new supernuk...,1


## Resampple Data secara acak

In [ ]:
real_data = df[df['label'] == 0].sample(n=10000, random_state=123)
fake_data = df[df['label'] == 1].sample(n=10000, random_state=123)

#menggabungkan data resample
sampled_data = pd.concat([real_data, fake_data])
sampled_data.shape



(20000, 5)

In [ ]:
df.to_csv('/content/drive/MyDrive/Tugas SEM3/Pemrosesan Teks/UTS/data_resample1.csv',index=False)

## pemanggilan Data resample

In [ ]:
import pandas as pd
df = pd.read_csv('G:\\My Drive\\Tugas SEM3\\Pemrosesan Teks\\Project Pemro\\data_resample1.csv')
df.head()

,title_text,body_len,hasil_preprocessed_title_text,label,tokens
0,Health insurers quietly shape Obamacare replac...,5218,health insur quietli shape obamacar replac few...,0,"['health', 'insurer', 'quietly', 'shape', 'oba..."
1,Migrants Refuse To Leave Train At Refugee Camp...,2447,migrant refus leav train refuge camp hungarymi...,0,"['migrant', 'refuse', 'leave', 'train', 'refug..."
2,Manafort meets with Senate intelligence panel:...,533,manafo meet senat intellig panel spokesmanwash...,0,"['manafort', 'meet', 'senate', 'intelligence',..."
3,CBO Releases Score of Paul Ryan’s American Hea...,327,cbo releas score paul ryan american health car...,0,"['cbo', 'releases', 'score', 'paul', 'ryan', '..."
4,Conservatives Urge Sessions to Clean Out Obama...,2283,conserv urg session clean obama civil right di...,0,"['conservative', 'urge', 'session', 'clean', '..."


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Ekstraksi Fitur Menggunakan TF-IDF dan BOW

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

## Train Sendiri
import nltk
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm
nltk.download('punkt')

import spacy
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dhani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Inisialisasi CountVectorizer
vectorizer = CountVectorizer()
vectorizer1 = TfidfVectorizer()

X = vectorizer.fit_transform(df['hasil_preprocessed_title_text'])
X1 =vectorizer1.fit_transform(df['hasil_preprocessed_title_text'])

feature_names = vectorizer.get_feature_names_out()
feature_names1 = vectorizer1.get_feature_names_out()
# fitur = vectorizer.get_feature_names_out()
# print('vocabolary',fitur )


## Ekstraksi Fitur Menggunakan Word2Vect

In [ ]:
#Tokenize jika train w2v sendiri
df['token']= [word_tokenize(berita) for berita in tqdm(df['hasil_preprocessed_title_text'])]

  0%|          | 0/20000 [00:00<?, ?it/s]

100%|██████████| 20000/20000 [00:38<00:00, 524.24it/s]


In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'as', 'made', 'by', 'itself', 'became', 'whereafter', 'several', 'serious', 'via', 'seem', 'whole', 'himself', 'on', 'eleven', 'at', 'alone', 'four', 'a', 'toward', 'more', 'we', 'no', '’d', 'give', 'empty', 'among', 'sometime', 'but', 'towards', 'would', 'one', 'less', 'top', 'everything', 'doing', 'once', 'was', 'neither', 'my', 'those', 'hers', 'now', 'nevertheless', 'almost', 'them', 'will', 'yet', 'should', 'it', 'me', 'through', 'off', 'put', 'become', '‘s', 'anything', 'yours', 'namely', 'until', 'you', 'using', 'could', 'whether', 'hereafter', 'ever', "'ve", 'about', 'everywhere', 'whereas', 'her', 'latter', 'herself', "'s", 'own', 'nowhere', '‘m', 'any', 'yourself', 'all', 'upon', 'full', 'may', 'over', 'none', 'noone', 'if', 'together', 'eight', 'mostly', 'say', 'hundred', 'ca', 'throughout', 'seeming', 'which', '‘ve', 'anywhere', 'without', 'beyond', 'afterwards', 'anyhow', 'cannot', 'indeed', 'had', 'yourselves', 'side', 'either', 'also', 'amount', 'or', 'within', 'somewhe

In [ ]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

df['tokens'] = tqdm(df['title_text'].apply(spacy_tokenizer))

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
df.to_csv('G:\\My Drive\\Tugas SEM3\\Pemrosesan Teks\\Project Pemro\\data_resample1.csv',index=False)

## Download Pretrain Model

In [ ]:
pip install gensim

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-100')
model.save('/content/drive/MyDrive/Tugas SEM3/Pemrosesan Teks/Project Pemro/glovetwitter.w2v')

## Building Model dan training model

In [ ]:
from gensim.models import Word2Vec
# model =  Word2Vec(df.tokens, vector_size=128, window=5, min_count = 3, epochs = 100)

model.save("/content/drive/MyDrive/Tugas SEM3/Pemrosesan Teks/Project Pemro/W2V_model")

##Load Model W2V

In [ ]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load('G:\\My Drive\\Tugas SEM3\\Pemrosesan Teks\\Project Pemro\\Modelgab')
# model = Word2Vec.load('/content/drive/MyDrive/Tugas SEM3/Pemrosesan Teks/Project Pemro/W2V_model').wv
# model.train(df.tokens,total_examples=len(df),epochs=40)

In [ ]:
token

Output hidden; open in https://colab.research.google.com to view.

Menambahkan kosakata baru pada Model W2V

In [ ]:
# Tambahkan kata-kata baru ke model pretrained
for sentence in tqdm(token, desc="Updating Model", unit="sentence"):
    for word in sentence:
        if word not in model:
            model[word] = np.random.rand(model.vector_size)

# Simpan model yang telah diperbarui
model.save('G:\\My Drive\\Tugas SEM3\\Pemrosesan Teks\\Project Pemro\\Modelgab')

Updating Model: 100%|██████████| 20000/20000 [00:06<00:00, 2861.62sentence/s]


In [ ]:
model.index_to_key

NameError: ignored

In [ ]:
model.vector_size

300

In [ ]:
model['risk']

array([-6.39648438e-02,  5.88378906e-02,  8.69140625e-02,  3.20312500e-01,
       -1.29882812e-01,  7.22656250e-02,  1.76757812e-01,  2.16796875e-01,
        2.87109375e-01, -1.69677734e-02, -1.07421875e-01, -5.00000000e-01,
        5.12695312e-02,  6.49414062e-02, -3.37890625e-01,  1.01074219e-01,
       -3.80859375e-01,  4.32128906e-02, -4.40597534e-04, -2.44140625e-01,
        1.37695312e-01, -4.82421875e-01, -2.10937500e-01,  2.18750000e-01,
        1.04003906e-01,  2.12890625e-01, -2.39257812e-02,  1.30859375e-01,
       -1.92382812e-01, -2.03857422e-02,  2.22656250e-01,  3.17382812e-02,
        8.78906250e-02, -1.45507812e-01, -8.54492188e-02, -1.00097656e-01,
        2.39257812e-01, -6.39648438e-02, -1.30859375e-01, -5.85937500e-02,
        7.08007812e-02,  2.23388672e-02, -5.02929688e-02, -7.66601562e-02,
       -9.71679688e-02, -1.87500000e-01, -2.86865234e-02,  1.19628906e-01,
        6.05468750e-02,  1.71875000e-01, -2.06054688e-01,  5.15747070e-03,
       -1.92382812e-01, -

In [ ]:
tqdm.pandas()
def sent_vec(sent):
    vector_size = model.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in model:
            ctr += 1
            wv_res += model[w]
    wv_res = wv_res/ctr
    return wv_res

df['vec'] = tqdm(df['tokens'].apply(sent_vec))

100%|██████████| 20000/20000 [00:00<00:00, 1577903.43it/s]


In [ ]:
import numpy as np

def mean_sent_vec(df_token, w2v_model):
    vec = [w2v_model[word] for word in df_token if word in w2v_model]
    sent_vec = np.mean(vec, axis=0) if vec else np.zeros_like(w2v_model.vector_size)
    return sent_vec

def norm_mean_sent_vec(df_token, w2v_model):
    vec = [w2v_model[word] for word in df_token if word in w2v_model]
    norm_vec = [vecs / np.linalg.norm(vecs) for vecs in vec if np.linalg.norm(vecs) > 0]
    sent_vec = np.mean(norm_vec, axis=0) if norm_vec else np.zeros_like(w2v_model.vector_size)
    return sent_vec

# Contoh penggunaan
df['vec'] = [norm_mean_sent_vec(sentence, model) for sentence in df.tokens]


In [ ]:
df.head()

,title_text,body_len,hasil_preprocessed_title_text,label,tokens,token,vec
0,Health insurers quietly shape Obamacare replac...,5218,health insur quietli shape obamacar replac few...,0,"['health', 'insurer', 'quietly', 'shape', 'oba...","[health, insur, quietli, shape, obamacar, repl...","[0.028409777, 0.0044648647, -0.009509044, 0.03..."
1,Migrants Refuse To Leave Train At Refugee Camp...,2447,migrant refus leav train refuge camp hungarymi...,0,"['migrant', 'refuse', 'leave', 'train', 'refug...","[migrant, refus, leav, train, refuge, camp, hu...","[0.028641444, 0.004985997, -0.011795795, 0.033..."
2,Manafort meets with Senate intelligence panel:...,533,manafo meet senat intellig panel spokesmanwash...,0,"['manafort', 'meet', 'senate', 'intelligence',...","[manafo, meet, senat, intellig, panel, spokesm...","[0.030697802, 4.6747467e-05, -0.010222687, 0.0..."
3,CBO Releases Score of Paul Ryan’s American Hea...,327,cbo releas score paul ryan american health car...,0,"['cbo', 'releases', 'score', 'paul', 'ryan', '...","[cbo, releas, score, paul, ryan, american, hea...","[0.036365375, -0.0017120375, -0.017321283, 0.0..."
4,Conservatives Urge Sessions to Clean Out Obama...,2283,conserv urg session clean obama civil right di...,0,"['conservative', 'urge', 'session', 'clean', '...","[conserv, urg, session, clean, obama, civil, r...","[0.029520774, 0.007132972, -0.012015657, 0.031..."


In [ ]:
similar_words = model.similar_by_word('trump', topn=5)
similar_words

[('donald', 0.7120490074157715),
 ('rogers', 0.6363762617111206),
 ('buffett', 0.6324752569198608),
 ('birther', 0.6219918131828308),
 ('bloomberg', 0.6092247366905212)]

In [ ]:
df.head()

,title_text,body_len,hasil_preprocessed_title_text,label,tokens,vec
0,Health insurers quietly shape Obamacare replac...,5218,health insur quietli shape obamacar replac few...,0,"[health, insur, quietli, shape, obamacar, repl...","[-0.020758405808479555, 0.02905586169612023, 0..."
1,Migrants Refuse To Leave Train At Refugee Camp...,2447,migrant refus leav train refuge camp hungarymi...,0,"[migrant, refus, leav, train, refuge, camp, hu...","[-0.03746505332204093, 0.06552155249941666, 0...."
2,Manafort meets with Senate intelligence panel:...,533,manafo meet senat intellig panel spokesmanwash...,0,"[manafo, meet, senat, intellig, panel, spokesm...","[-0.08503655327690972, 0.06862013075086805, 0...."
3,CBO Releases Score of Paul Ryan’s American Hea...,327,cbo releas score paul ryan american health car...,0,"[cbo, releas, score, paul, ryan, american, hea...","[-0.0761670564350329, 0.10274947317023027, 0.0..."
4,Conservatives Urge Sessions to Clean Out Obama...,2283,conserv urg session clean obama civil right di...,0,"[conserv, urg, session, clean, obama, civil, r...","[-0.023430870089468477, 0.0551246409853473, 0...."


In [ ]:
len(df['tokens'][0])

5539

## Clustering

In [ ]:
from sklearn.cluster import KMeans

# Inisialisasi model K-Means dengan jumlah cluster yang diinginkan
num_clusters = 2  # Gantilah dengan jumlah cluster yang sesuai
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Melakukan K-Means pada data
kmeans.fit(df['vec'].to_list())  # Gantilah 'data_matrix' dengan vektor TF-IDF atau BoW

# Menyimpan label cluster untuk setiap dokumen
df['Hasil_cluster'] = kmeans.labels_


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
print(df.label.value_counts())
print(df.Hasil_cluster.value_counts())

0    10000
1    10000
Name: label, dtype: int64
1    10933
0     9067
Name: Hasil_cluster, dtype: int64


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(df['vec'].to_list(), kmeans.labels_)
silhouette_avg #semakin mendekati 1 semakin baik

0.09161831585090417

In [ ]:
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(df['label'], kmeans.labels_) #semakin mendekati 1 semakin baik
ari

-4.1159274013053665e-05

## Split Data


In [ ]:
from sklearn.model_selection import train_test_split

# Data Anda (gantilah ini dengan data sesuai dengan format Anda)
X = df['vec'].to_list() # Variabel independen
y = df['label'].to_list()  # Variabel target

# Membagi data menjadi data pelatihan (80%) dan data pengujian (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)



## RF W2V

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76      1985
           1       0.77      0.74      0.75      2015

    accuracy                           0.75      4000
   macro avg       0.75      0.75      0.75      4000
weighted avg       0.75      0.75      0.75      4000



## GOOGLE

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83      1985
           1       0.84      0.84      0.84      2015

    accuracy                           0.83      4000
   macro avg       0.83      0.83      0.83      4000
weighted avg       0.83      0.83      0.83      4000



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Membuat model Random Forest
rf = RandomForestClassifier()

# Mendefinisikan grid parameter yang ingin dioptimalkan
param_grid = {
    'n_estimators': [100, 300, 500],  # Coba beberapa jumlah pohon
    'max_depth': [None, 5, 10, 15],  # Coba beberapa kedalaman maksimum
    'min_samples_split': [2, 5, 10]  # Coba beberapa nilai min_samples_split
}

# Mencari parameter terbaik menggunakan GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
print("Parameter terbaik:", grid_search.best_params_)

# Melihat performa model terbaik
print("Akurasi terbaik:", grid_search.best_score_)



## Logistic regresion W2V

In [ ]:
from sklearn.linear_model import LogisticRegression
# Inisialisasi model Regresi Logistik
logistic_regression = LogisticRegression()

# Latih model pada data pelatihan
logistic_regression.fit(X_train, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report
y_pred2 = logistic_regression.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.63      0.72      0.67      1985
           1       0.68      0.59      0.63      2015

    accuracy                           0.65      4000
   macro avg       0.66      0.65      0.65      4000
weighted avg       0.66      0.65      0.65      4000



## GOOGLE

In [ ]:
from sklearn.metrics import classification_report
y_pred2 = logistic_regression.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      1985
           1       0.84      0.84      0.84      2015

    accuracy                           0.84      4000
   macro avg       0.84      0.84      0.84      4000
weighted avg       0.84      0.84      0.84      4000



## SVM W2V

In [ ]:
from sklearn.svm import SVC
# Inisialisasi model SVM (SVC)
svm_classifier = SVC(kernel='linear')

# Latih model pada data pelatihan
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
from sklearn.metrics import classification_report
y_pred2 = svm_classifier.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.60      0.84      0.70      1985
           1       0.74      0.44      0.55      2015

    accuracy                           0.64      4000
   macro avg       0.67      0.64      0.62      4000
weighted avg       0.67      0.64      0.62      4000



## GOOGLE

In [ ]:
from sklearn.metrics import classification_report
y_pred2 = svm_classifier.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      1985
           1       0.85      0.85      0.85      2015

    accuracy                           0.85      4000
   macro avg       0.85      0.85      0.85      4000
weighted avg       0.85      0.85      0.85      4000



##KNN W2V

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)  # misalnya menggunakan 5 tetangga terdekat
knn.fit(X_train, y_train)  # Latih model dengan data latih


KNeighborsClassifier(n_neighbors=3)

In [ ]:
from sklearn.metrics import classification_report
y_pred2 = knn.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.66      0.72      0.69      1985
           1       0.70      0.64      0.67      2015

    accuracy                           0.68      4000
   macro avg       0.68      0.68      0.68      4000
weighted avg       0.68      0.68      0.68      4000



## GOOGLE

In [ ]:
from sklearn.metrics import classification_report
y_pred2 = knn.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.74      0.88      0.80      1985
           1       0.85      0.69      0.77      2015

    accuracy                           0.79      4000
   macro avg       0.80      0.79      0.78      4000
weighted avg       0.80      0.79      0.78      4000

